In [1]:
import pandas as pd
import numpy as np

from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import eli5

In [2]:
df_train = pd.read_hdf("../input/df.train.h5")
df_test = pd.read_hdf("../input/df.test.h5")

df = pd.concat([df_train, df_test])

df['price_value'] = df['price_value'].str.replace(' ', '').str.replace(',', '.').astype('float')
df.drop([106447], inplace=True) #delete outlier

def netto_brutto(row):
    if 'Netto' in row['price_details']:
        return row['price_value'] * 1.23
    return row['price_value']

df.loc[~df['price_value'].isnull(), 'price_value'] = df.loc[~df['price_value'].isnull(),['price_details', 'price_value']].apply(netto_brutto, axis=1)


In [3]:
#prepering 'features' column
features_list = df['features'].tolist()
features = []
for x in features_list:
    for y in x:
        if y.lower() not in features:
            features.append(y.lower())
            
for feature in features:
    df[feature] = df['features'].map(lambda x: 1 if feature in [i.lower() for i in x] else 0)

In [4]:
#prepering 'breadcrumb' column
df['type'] = df['breadcrumb'].str[0]
df['brand'] = df['breadcrumb'].str[1]
df['model'] = df['breadcrumb'].str[2]
df['version'] = df['breadcrumb'].map(lambda x: x[-1] if len(x)>3 else np.nan)

In [5]:
#prepering 'offer_params' column
params = df['offer_params'].apply(pd.Series)

pairs = [('Marka pojazdu', 'Make'), ('Metalik', 'Metallic'), ('Pojemność skokowa', 'Engine capacity'), ('Wersja', 'Version'),
        ('Bezwypadkowy', 'No accident'), ('Przebieg', 'Mileage'), ('Serwisowany w ASO', 'Service record'),
        ('Liczba drzwi', 'Door count'), ('Rodzaj paliwa', 'Fuel type'), ('Liczba miejsc', 'Nr of seats'),
         ('Skrzynia biegów', 'Gearbox'), ('Napęd', 'Transmission'), ('Moc', 'Engine power'), ('Model pojazdu', 'Model'),
        ('Kolor', 'Color'), ('Kraj pochodzenia', 'Country of origin'), ('Typ', 'Body type'), ('Rok produkcji', 'Year'),
        ('Akryl (niemetalizowany)', 'Acrylic'), ('Możliwość finansowania', 'Financing option'),
         ('Zarejestrowany w Polsce', 'Registered in Poland'), ('Pierwszy właściciel', 'Original owner'),
        ('Pierwsza rejestracja', 'First registration'), ('Perłowy', 'Pearl'), ('Kod Silnika', 'Engine Code'),
        ('Filtr cząstek stałych', 'Particle filter'), ('Leasing', 'Leasing concession'), ('Uszkodzony', 'Damaged'),
        ('Emisja CO2', 'CO2 emissions'), ('Miesięczna rata', 'Monthly payment value')]

for pl, en in pairs:
    params['{}_new'.format(en)] = params[pl].combine_first(params[en])
    params.drop(columns=[pl, en], inplace=True)
    
params = params.fillna(-1) # change NaN values to -1 for easier transformation and ML algorithms
if "Oferta od" not in df:
    df = pd.concat([df, params], axis=1)

In [6]:
voivodships = ['dolnośląskie', 'kujawsko-pomorskie', 'lubelskie', 'lubuskie', 'łódzkie' , 'małopolskie', 'mazowieckie', 
               'wielkopolskie', 'opolskie', 'podkarpackie', 'podlaskie', 'śląskie', 'świętokrzyskie', 
               'warmińsko-mazurskie', 'zachodniopomorskie', 'pomorskie']
def voivodship(x):
    for i in voivodships:
        if i in x.lower():
            return i
    return ''
df['voivodship'] = df['seller_address'].map(voivodship)

In [7]:
#numeric features
df['Year_new'] = df['Year_new'].astype(int)
df['Nr of seats_new'] = df['Nr of seats_new'].astype(int)

df['Engine capacity_new'] = df['Engine capacity_new'].map(lambda x: str(x).split('cm3')[0].replace(' ', '')).astype(int)

df['Mileage_new'] = df['Mileage_new'].map(lambda x: str(x).split('km')[0].replace(' ', '')).astype(int)

df['Door count_new'] = df['Door count_new'].astype(int)

df['Engine power_new'] = df['Engine power_new'].map(lambda x: str(x).replace('KM', '').replace('HP', '').replace(' ', '')).astype(int)

df['First registration_new'] = df['First registration_new'].map(lambda x: str(x).split(' ')[-1]).astype(int)

df['CO2 emissions_new'] = df['CO2 emissions_new'].map(lambda x: str(x).split('g/km')[0].replace(' ', '')).astype(int)

df.loc[df['Door count_new']==30, 'Door count_new'] = 3
df.loc[df['Door count_new']==1, 'Door count_new'] = -1
df.loc[df['Door count_new']==7, 'Door count_new'] = -1
df.loc[df['Door count_new']==8, 'Door count_new'] = -1
df.loc[df['Door count_new']==9, 'Door count_new'] = -1
df.loc[df['Nr of seats_new']==1, 'Nr of seats_new'] = -1

In [8]:
expensive_feats = ['gniazdo sd', 'czujniki parkowania przednie', 'asystent parkowania', 'światła led', 'klimatyzacja czterostrefowa',
                  'system start-stop', 'tempomat aktywny', 'regulowane zawieszenie', 'kamera cofania', 'podgrzewane tylne siedzenia',
                  'czujnik martwego pola', 'asystent pasa ruchu', 'hud (wyświetlacz przezierny)', 'łopatki zmiany biegów']
df['has_expensive_feat'] = df[expensive_feats].apply(lambda x: x.values.any(), axis=1).astype(int)

expensive_brand = ['Lamborghini', 'Ferrari', 'McLaren', 'Maybach', 'Bentley', 'Tesla', 'Aston Martin', 'Rolls-Royce', 'Maserati', 'Porsche']
df['expensive_brand'] = df['brand'].map(lambda x: x in expensive_brand).astype(int)

df['vintage'] = np.where((df['Year_new']<2000) & (df['Zarejestrowany jako zabytek']=='Tak'), 1, 0)

df['damage_accident'] = ((df['No accident_new']=='Tak') & (df['Damaged_new']==-1)).astype(int)

df['transmission_gearbox'] = (((df['Transmission_new'] == '4x4 (dołączany automatycznie)') | (df['Transmission_new'] == '4x4 (stały)')) & ((df['Gearbox_new'] == 'Automatyczna bezstopniowa (CVT)') | (df['Gearbox_new'] == 'Automatyczna hydrauliczna (klasyczna)') | (df['Gearbox_new'] == 'Automatyczna dwusprzęgłowa (DCT, DSG)'))).astype(int)

df['nr_of_features'] = df['features'].map(lambda x: len(x))

df['nr_expensive_feats'] = df[expensive_feats].apply(lambda x: x.sum(), axis=1)

df['transmission_fuel'] = (((df['Transmission_new'] == '4x4 (dołączany automatycznie)') | (df['Transmission_new'] == '4x4 (stały)')) & (df['Fuel type_new'] != 'Benzyna+LPG')).astype(int)

df['gearbox_fuel'] = (((df['Gearbox_new'] == 'Automatyczna bezstopniowa (CVT)') & ((df['Fuel type_new'] == 'Benzyna') | (df['Fuel type_new'] == 'Elektryczny') | (df['Fuel type_new'] == 'Hybryda'))) |\
                     ((df['Gearbox_new'] == 'Automatyczna dwusprzęgłowa (DCT, DSG)') & ((df['Fuel type_new'] == 'Benzyna') | (df['Fuel type_new'] == 'Diesel') | (df['Fuel type_new'] == 'Hybryda'))) |\
                     ((df['Gearbox_new'] == 'Automatyczna hydrauliczna (klasyczna)') & ((df['Fuel type_new'] == 'Benzyna') | (df['Fuel type_new'] == 'Diesel') | (df['Fuel type_new'] == 'Elektryczny') | (df['Fuel type_new'] == 'Hybryda'))) |\
                     ((df['Gearbox_new'] == 'Manualna') & (df['Fuel type_new'] == 'Hybryda')) |\
                     ((df['Gearbox_new'] == '-1') & (df['Fuel type_new'] == 'Elektryczny'))).astype(int)


In [9]:
def reset_outlires(df, feat, prc=99):
    cut_value = np.percentile(df[feat], prc)
    
    return df[feat].map(lambda x: x if x < cut_value else -1)

def replace_capacity(row):
    if row > 1000000:
        return int(row/1000)
    elif row > 70000:
        return int(row/100)
    elif row > 9000:
        return int(row/10)
    
    return row

def replace_power(row):
    if row > 100000:
        return int(row/1000)
    elif row > 5000:
        return int(row/100)
    
    return row

def swap_engine(row):
    if (row['Engine power_new_rep'] > 800) and (row['Engine power_new_rep'] > row['Engine capacity_new_rep']) and (row['Engine capacity_new_rep'] > 0) and (row['Engine capacity_new_rep'] < 600):
        return pd.Series([row['Engine power_new_rep'], row['Engine capacity_new_rep']])
    return pd.Series([row['Engine capacity_new_rep'], row['Engine power_new_rep']])

def replace_capacity_up(row):
    if row['Engine capacity_new_rep'] < 10 and row['Engine capacity_new_rep'] != -1:
        return row['Engine capacity_new_rep'] * 1000
    elif row['Engine capacity_new_rep'] > 9 and row['Engine capacity_new_rep'] < 40:
        return row['Engine capacity_new_rep'] * 100
    elif (row['Engine capacity_new_rep'] > 39 and row['Engine capacity_new_rep'] < 300) \
    or (row['Engine capacity_new_rep'] > 299 and row['Engine capacity_new_rep'] < 700 and row['Engine power_new_rep'] > 120):
        return row['Engine capacity_new_rep'] * 10
    return row['Engine capacity_new_rep']

def replace_power_up(row):
    if row['Engine power_new_rep'] > 850 :
        return row['Engine power_new_rep'] / 10
    elif row['Engine power_new_rep'] < 70 and row['Engine capacity_new_rep'] > 2000:
        return -1
    return row['Engine power_new_rep']

In [10]:
df['Engine capacity_new_rep'] = df['Engine capacity_new'].map(replace_capacity)
df['Engine power_new_rep'] = df['Engine power_new'].map(replace_power)
# swap capacity with power under conditions.
df[['Engine capacity_new_rep', 'Engine power_new_rep']] = df[['Engine capacity_new_rep', 'Engine power_new_rep']].apply(swap_engine, axis=1)
df['Engine capacity_new_rep'] = df[['Engine capacity_new_rep', 'Engine power_new_rep']].apply(replace_capacity_up, axis=1)
df['Engine power_new_rep'] = df[['Engine capacity_new_rep', 'Engine power_new_rep']].apply(replace_power_up, axis=1).astype(int)

df['Mileage_new_prc'] = reset_outlires(df, 'Mileage_new', prc=99.926)
df['Mileage_new_prc'] = df[['Mileage_new_prc', 'Year_new']].apply(lambda x: -1 if x['Mileage_new_prc']<101 and x['Year_new']<2017 else x['Mileage_new_prc'], axis=1)

In [11]:
obj_feats = df.select_dtypes(object).columns

for feat in obj_feats:
    some_value = df[feat].values[0]
    if isinstance(some_value, list): continue
    if isinstance(some_value, dict): continue
    
    df['{}_cat'.format(feat)] = df[feat].factorize()[0]
    
feats_int = df.select_dtypes('int64').columns

In [12]:
feats_drop = ['id', 'Kategoria_cat', 'type_cat', 'Make_new_cat', 'Model_new_cat', 'Oferta od_cat', 'price_currency_cat',
              'price_details_cat', 'Liczba pozostałych rat_cat', 'Opłata początkowa_cat', 'Wartość wykupu_cat', 
              'Monthly payment value_new_cat', 'VIN_cat', 'VAT discount_cat', 'Matowy', 'VAT free_cat', 'Zarejestrowany jako zabytek_cat',
             'Mileage_new', 'Engine capacity_new', 'Engine power_new']

def get_feat(feats_int, feats_drop, feats_add=[]):
#     print(feats_drop)
    feats = [feat for feat in feats_int if feat not in feats_drop]
    return feats + feats_add

In [31]:
params = {'max_depth': 5,
            'colsample_bytree': 0.9,
            'learning_rate': 0.2,
            'subsample': 0.92,
            'random_state': 21,
            'n_estimators': 100,
            }

def validate_model(df, feats, params, eli=True):
    df_train = df[~df['price_value'].isnull()].copy()
    
    X = df_train[feats].values
    y = df_train['price_value'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=21)

    model = XGBRegressor(**params)
    model.fit(X_train, np.log(y_train))
    y_pred = model.predict(X_test)
    score = mean_absolute_error(y_test, np.exp(y_pred))
    print('MAE: {}'.format(score))

    if eli:
        model.fit(X, y)
        display(eli5.show_weights(model, feature_names=feats))
        
def submit(df, feats, params):
    train = df[~df['price_value'].isnull()].copy()
    test = df[df['price_value'].isnull()]
    test = test.drop('price_value', axis=1)
    
    X = train[feats].values
    y = train['price_value'].values
    X_test = test[feats].values
    
    model = XGBRegressor(**params)
    model.fit(X, np.log(y))
    y_pred = np.exp(model.predict(X_test))
    
    test['price_value'] = y_pred
    test[ ['id', 'price_value'] ].to_csv("../output/xgb_1.csv", index=False)

In [14]:
feats = get_feat(feats_int, feats_drop)
validate_model(df, feats, params)

MAE: 7882.886367858067


Weight,Feature
0.1129,Year_new
0.0844,Engine capacity_new_rep
0.0739,Engine power_new_rep
0.0614,Transmission_new_cat
0.0561,podgrzewane przednie siedzenia
0.0476,expensive_brand
0.0473,transmission_gearbox
0.0450,damage_accident
0.0357,nr_expensive_feats
0.0307,Fuel type_new_cat


In [34]:
params = {'max_depth': 6,
            'colsample_bytree': 0.85,
            'learning_rate': 0.15,
            'subsample': 0.90,
            'random_state': 21,
            'n_estimators': 800}
feats = get_feat(feats_int, feats_drop)
validate_model(df, feats, params, eli=False)

MAE: 6560.56969450707


In [35]:
params = {'max_depth': 6,
            'colsample_bytree': 0.85,
            'learning_rate': 0.15,
            'subsample': 0.9,
            'random_state': 21,
            'n_estimators': 800}
feats = get_feat(feats_int, feats_drop)
submit(df, feats, params)